In [1]:
import torch
import torch.nn as nn
import pandas as pd
import pickle


In [2]:
enc_path = r'C:\Users\user\metrosafe-1\model\test\label_encodersline1.pkl'
with open(enc_path, 'rb') as f:
    encoders = pickle.load(f)

# encoders 예시: {'호선': LabelEncoder, '역명': LabelEncoder, ...}


In [3]:
class SubwayPredictor(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )
    def forward(self, x):
        return self.layers(x)

# 입력 변수 개수 및 클래스 수(=칸 수) 맞춤
input_dim = 5         # ['호선','역명','요일','시간','상하행']
num_classes = 10      # 칸 수에 맞게 (10으로 예시)

model_path = r'C:\Users\user\metrosafe-1\model\test\subway_predictorline1.pt'

model = SubwayPredictor(input_dim, num_classes)
model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.eval()


SubwayPredictor(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [10]:
# 저장된 encoders 딕셔너리에서 각 컬럼별 classes_ 출력
for col in ['호선', '역명', '요일', '상하행']:
    print(f"컬럼명: {col}")
    print(encoders[col].classes_)
    print("="*40)


컬럼명: 호선
['1호선']
컬럼명: 역명
['동대문역' '동묘앞역' '서울역' '시청역' '신설동역' '제기동역' '종각역' '종로3가역' '종로5가역' '청량리역']
컬럼명: 요일
['MON' 'SAT']
컬럼명: 상하행
[0 1]


In [12]:
sample = pd.DataFrame({
    '호선':    ['1호선'],          # 그대로 사용
    '역명':    ['서울역'],         # 그대로 사용
    '요일':    ['MON'],           # 반드시 'MON' 또는 'SAT'만! (한글 '평일', '주말' X)
    '시간':    [9],               # 그냥 숫자
    '상하행':  [0]                # 숫자 0(상행), 1(하행)
})


In [13]:
# 요일 한글→영문 변환 필요시 먼저 수행
요일_map = {'평일': 'MON', '주말': 'SAT'}
if sample['요일'].iloc[0] in 요일_map:
    sample['요일'] = sample['요일'].map(요일_map)

# 상하행이 한글로 들어오는 경우 숫자로 변환 필요
상하행_map = {'상행': 0, '하행': 1}
if str(sample['상하행'].iloc[0]) in 상하행_map:
    sample['상하행'] = sample['상하행'].map(상하행_map)

# 나머지 컬럼(호선, 역명, 요일)은 LabelEncoder로 transform
for col in ['호선', '역명', '요일']:
    sample[col] = encoders[col].transform(sample[col])

# 이제 tensor 변환
X_input = torch.tensor(sample.values, dtype=torch.float32)


In [14]:
with torch.no_grad():
    logits = model(X_input)         # (배치, 클래스수) shape
    pred = torch.argmax(logits, dim=1).numpy()
    print("예측된 가장 한적한 칸 인덱스:", pred)     # 예: [3]


예측된 가장 한적한 칸 인덱스: [0]
